In [1]:
import sys
import os
import dspy 
from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

lm_gpt35 = dspy.LM('gpt-3.5-turbo', temperature=0.8, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
lm_gpt4omin = dspy.LM('gpt-4o-mini', temperature=0.9, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
dspy.configure(lm=lm_gpt4omin)

lm = lm_gpt4omin
lm('hello there')


Getting keys from environment variables


['Hello! How can I assist you today?']

In [2]:
lm_gpt4omin('Hows the weather')

['I don’t have real-time data access to check the current weather. You can use a weather website or app for the latest updates on the weather in your area. If you let me know your location, I can suggest how to find that information!']

In [ ]:
lm_gpt4omin.history

[{'prompt': 'hello there',
  'messages': None,
  'kwargs': {},
  'response': ModelResponse(id='chatcmpl-CC9lWbC3WhS9UlxSIstQ37mZJYeoR', created=1757013510, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_560af6e559', choices=[Choices(finish_reason='stop', index=0, message=Message(content='Hello! How can I assist you today?', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=9, prompt_tokens=9, total_tokens=18, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default', cache_hit=None),
  'outputs': ['Hello! How can I assist you today?'],
  'usage': {'completion_tokens': 9,
   

In [5]:
from datetime import datetime

print(datetime.now())

2025-09-04 21:22:01.352007


In [7]:
len(lm.history)
lm.history[-1].keys()

dict_keys(['prompt', 'messages', 'kwargs', 'response', 'outputs', 'usage', 'cost', 'timestamp', 'uuid', 'model', 'response_model', 'model_type'])

In [12]:
toxicity = dspy.Predict(
    dspy.Signature(
        "comment -> toxic: bool", 
        instructions="Mark as 'toxic' if the comment includes insults, harrasment or sarcastic comments"
    )
)

toxicity(comment="You are awesome").toxic

False

In [ ]:
choose_meal = dspy.Predict(
    dspy.Signature(
    "meal_suggestion -> description",
    instructions="Given a meal suggestion come up with a meal and desribe it in 20 words, make it sound enticing without going overboard"
))

md(choose_meal(meal_suggestion="South African").description)


Savor a flavorful Bobotie, a spiced minced meat dish topped with egg custard, served with fragrant rice and chutney.

In [ ]:
sentence = "Today is rainy and gloomy"

classify = dspy.Predict("sentence -> sentiment: str") # change type: int, float, str, bool
classify(sentence=sentence).sentiment

'negative'

In [22]:
argh = "Let's go to a bar tonight and listen to a band"
pirate_speak = dspy.Predict("normal_speak -> pirate_speak")
pirate_speak(normal_speak=argh).pirate_speak

'Arrr, let’s venture to a tavern tonight and lend an ear to a crew of musicians!'

In [27]:
question = "What colour are plant leaves?"
reason_question = dspy.Predict("question -> answer")
md(reason_question(question=question))

Prediction(  
    answer='Plant leaves are primarily green due to the presence of chlorophyll, which is essential for photosynthesis. However, leaves can also be other colors, such as red, purple, yellow, or even variegated, depending on the plant species and environmental factors like sunlight and soil nutrients.'  
)

In [33]:
from typing import Literal

class Emotion(dspy.Signature):
    """Classify emotion"""
    sentence: str = dspy.InputField()
    sentiment: Literal["sadness", "joy", "love", "fear", "surprised"]  = dspy.OutputField()

classify = dspy.Predict(Emotion)
classify(sentence="rainy day")

Prediction(
    sentiment='sadness'
)

https://dspy.ai/learn/programming/signatures/

In [38]:
class CheckCitationOfFaithfulness(dspy.Signature):
    """Verify that the text is based on the provided context."""
    context: str = dspy.InputField(desc="Facts here are assumed to be true")
    text: str = dspy.InputField()
    faithfulness: float = dspy.OutputField()
    evidence: dict[str, list[str]] = dspy.OutputField(desc="Supporting evidence for claims")
    counter_argument: dict[str, dict[str, str]] = dspy.OutputField(desc="Give a counter argument to this")

context = """
The 21-year-old made seven appearances for the Hammers and netted his only 
goal for them in a Europa League qualification round match against Andorran side 
FC Lustrains last season. Lee had two loan spells in League One last term, 
with Blackpool and then Colchester United. He scored twice for the U's but 
was unable to save them from relegation. 
The length of Lee's contract with the promoted Tykes has not been revealed. 
Find all the latest football transfers on our dedicated page.
"""
text = "Lee scored 3 goals for the Colchester United"

faithfulness = dspy.ChainOfThought(CheckCitationOfFaithfulness)
faithfulness(context=context, text=text)


Prediction(
    reasoning='The text claims that Lee scored 3 goals for Colchester United, but the context states that he scored twice for them. Therefore, the statement in the text is incorrect based on the provided context.',
    faithfulness=0.0,
    evidence={'goals_scored': ["Lee scored twice for the U's", 'unable to save them from relegation']},
    counter_argument={'text_inaccuracy': {'claim': 'Lee scored 3 goals for Colchester United', 'correction': 'Lee actually scored 2 goals for Colchester United'}}
)

In [ ]:
# factory = LlmClientFactory(settings)
# db = factory.get_dspy_lm_chat_connecting_to_internal_databricks_model()

TypeError: LlmClientFactory.get_dspy_lm_chat_connecting_to_internal_databricks_model() missing 1 required positional argument: 'model'

In [41]:
class PictureSignature(dspy.Signature):
    """Output the dog breed of the picture"""
    image_1: dspy.Image = dspy.InputField(desc="An image of a dog")
    answer: str = dspy.OutputField(desc="The dog breed of the dog in the picture")

picture = dspy.Predict(PictureSignature)
picture(image_1=dspy.Image.from_url("https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U"))

Prediction(
    answer='Labrador Retriever'
)